<a href="https://colab.research.google.com/github/yuta4u2/test1/blob/master/GrowthAccounting_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
countries = ["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"]
data = pwt1001.loc[pwt1001['country'].isin(countries)][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'labsh', 'avh', 'rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['hours'] = data['emp'] * data['avh']
data['y_L'] = np.log(data['rgdpna'] / data['hours'])  # ln(Y/L)
data['k_L'] = np.log(data['rkna'] / data['hours'])  # ln(K/L)
data['tfp'] = np.log(data['rtfpna']) # ln(A)

# Order by year
data = data.sort_values("year")

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates 
data["Growth_rate"] = grouped_data['y_L'].diff() * 100
data["Capital_Growth"] = grouped_data['k_L'].diff() * 100
data["Capital_Deepning"] = data["Capital_Growth"] * data['alpha']
data["TFP_Growth"] = grouped_data['tfp'].diff() * 100


data = data.dropna()

summary = data.groupby('countrycode').agg({'Growth_rate': 'mean',
                                       'Capital_Deepning': "mean",
                                       'TFP_Growth': "mean",})
summary['TFP_Share'] = summary['TFP_Growth']/summary['Growth_rate']
summary['Capital_Share'] = summary['Capital_Deepning']/summary['Growth_rate']
summary = summary.reindex(columns=['Growth_rate', 'TFP_Growth', 'Capital_Deepning', 'TFP_Share', 'Capital_Share'])
summary.loc['Average'] = summary.mean()
summary

,Growth_rate,TFP_Growth,Capital_Deepning,TFP_Share,Capital_Share
countrycode,,,,,
CAN,1.117384,0.151360,0.713034,0.135459,0.638128
DEU,1.073874,0.358568,0.584524,0.333902,0.544313
FRA,1.087678,0.093816,0.666056,0.086254,0.612365
GBR,1.149943,0.230807,0.628362,0.200711,0.546429
ITA,0.285631,-0.772207,0.689669,-2.703511,2.414547
JPN,1.221666,0.142913,0.848135,0.116982,0.694244
USA,1.658237,0.675462,0.825947,0.407337,0.498088
Average,1.084916,0.125817,0.707961,-0.203266,0.849731
